In [1]:
import os
import shutil
import random

source_base_dir = '/kaggle/input/pix-og/Image_Processing'  


base_dir = '/kaggle/working/'


original_dir = os.path.join(source_base_dir, 'Original')
pixelated_dir = os.path.join(source_base_dir, 'Pixelated')


train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)


split_ratio = 0.8

def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)

    
    paired_files = [(file, file) for file in original_files if file in pixelated_files]

    random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]

    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
    
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))

split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

IMAGE_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/working/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary', 
    subset='training'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/working/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
)


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))


2024-07-03 10:18:11.144115: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 10:18:11.144309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 10:18:11.311652: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 14486 images belonging to 2 classes.
Found 3622 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [4]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


453/453 ━━━━━━━━━━━━━━━━━━━━ 457s 986ms/step - accuracy: 0.8700 - loss: 0.3019 - val_accuracy: 0.9144 - val_loss: 0.2164
Epoch 2/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.9266 - loss: 0.1908 - val_accuracy: 0.9373 - val_loss: 0.1702
Epoch 3/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.9373 - loss: 0.1564 - val_accuracy: 0.9304 - val_loss: 0.1824
Epoch 4/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 448s 983ms/step - accuracy: 0.9434 - loss: 0.1397 - val_accuracy: 0.9359 - val_loss: 0.1612
Epoch 5/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 533s 1s/step - accuracy: 0.9596 - loss: 0.1032 - val_accuracy: 0.9337 - val_loss: 0.1735
Epoch 6/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 472s 1s/step - accuracy: 0.9663 - loss: 0.0842 - val_accuracy: 0.9373 - val_loss: 0.1717
Epoch 7/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 472s 1s/step - accuracy: 0.9742 - loss: 0.0654 - val_accuracy: 0.9420 - val_loss: 0.1739
Epoch 8/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 505s 1s/step - accuracy: 0.9747 - loss: 0.0593 - val_accuracy:

In [6]:
model.save('/kaggle/working/pixel_classification_model.h5')

In [7]:
loss, accuracy = model.evaluate(test_generator)
print(f'Accuracy: {accuracy * 100:.2f}%')

114/114 ━━━━━━━━━━━━━━━━━━━━ 97s 848ms/step - accuracy: 0.9397 - loss: 0.1872
Accuracy: 93.76%
